In [25]:
# !pip install numba==0.48
# !pip install regain
# !pip install plotly

In [1]:
%matplotlib inline

import sys
import time
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance

sys.path.append('..')
from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import kkt_ADMM_SGL
from helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from helper.model_selection import single_grid_search


from regain.covariance import GraphicalLasso as rg_GL
#from regain.covariance.graphical_lasso_ import GraphicalLasso
# from regain.covariance import LatentTimeGraphicalLasso
# from regain.covariance import latent_graphical_lasso_, latent_time_graphical_lasso_

# Sample data

In [27]:
true_cov = np.array([[0.8, 0.0, 0.2, 0.0],
                     [0.0, 0.4, 0.0, 0.0],
                     [0.2, 0.0, 0.3, 0.1],
                     [0.0, 0.0, 0.1, 0.7]])

In [28]:
np.random.seed(243)
X = np.random.multivariate_normal(mean=[0, 0, 0, 0], cov=true_cov, size=100)

X_trans = X.T  #Regain and Sklearn transpose data during training
X_cov = np.cov(X_trans) #input for ADMM

X_cov.shape, X_trans.shape

((4, 4), (4, 100))

X is a randomly generated data which can serve as an input to sklearn and regain methods.
ADMM works with covarince matrix, so we have to extract it first.

# Model solution Z

In [29]:
Z = sk_GL(max_iter=1000, tol = 1e-10).fit(X)
Z.get_params()

{'alpha': 0.01,
 'assume_centered': False,
 'enet_tol': 0.0001,
 'max_iter': 1000,
 'mode': 'cd',
 'tol': 1e-10,
 'verbose': False}

In [30]:
Z.precision_

array([[ 1.53284181,  0.2366127 , -0.59705507,  0.22230787],
       [ 0.2366127 ,  2.26999954,  0.22522913,  0.05631177],
       [-0.59705507,  0.22522913,  3.0262301 , -0.28105847],
       [ 0.22230787,  0.05631177, -0.28105847,  1.61698823]])

# Import sklearn

In [31]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = np.linspace(1e-10, 1e-4, num=100)
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = sk_GL(tol=tol, max_iter = 1000).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [32]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z.precision_ - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [33]:
method_list = ["sklearn"]*len(accuracy_list)

In [41]:
str_tol = [str(x) for x in tol_list]

In [35]:
df_sk = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method':method_list,
                            'tol_rate': str_tol})
df_sk.head()

,time,accuracy,method,tol_rate
0,0.002336,0.999993,sklearn,0.0001
1,0.001507,0.999993,sklearn,1e-05
2,0.001051,0.999993,sklearn,1e-06
3,0.001518,1.000000,sklearn,1e-07


# Import Regain GL

In [36]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = np.linspace(1e-10, 1e-4, num=100)
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = rg_GL(alpha=0.01, rho=1.,tol=tol, rtol=1e-7, max_iter=1000, verbose=False).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [37]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z.precision_ - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [38]:
method_list = ["regain"]*len(accuracy_list)

In [39]:
df_rg = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method': method_list,
                            'tol_rate': str_tol})
df_rg.head()

,time,accuracy,method,tol_rate
0,0.002289,0.997916,regain,0.0001
1,0.001779,0.999697,regain,1e-05
2,0.002567,0.999976,regain,1e-06
3,0.002667,0.999999,regain,1e-07


# Import ADMM SGL

In [40]:
# Omega_0 = np.eye(len(X_cov))
Omega_0 = np.eye(len(X_cov))
lambda1 = 0.01

In [41]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
#     sol, info = ADMM_SGL(X_cov, lambda1, Omega_0, max_iter=1000, tol = tol, verbose = False, latent = False)
    sol, info = ADMM_SGL(X_cov, lambda1, Omega_0, max_iter=1000, tol = tol, verbose = False, latent = False)
    end = time.time()
    cov_list.append(sol['Omega'])
    precision_list.append(sol['Theta'])
    time_list.append(end-start)

ADMM terminated after 54 iterations with accuracy 0.0009581011918730356
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 77 iterations with accuracy 0.00012422398836316805
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 89 iterations with accuracy 4.2885779226089456e-05
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 92 iterations with accuracy 3.2875226093742374e-05
ADMM status: {'dual optimal', 'primal optimal'}


In [42]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z.precision_ - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [43]:
method_list = ["admm"]*len(accuracy_list)

In [44]:
df_admm = pd.DataFrame(data={'time': time_list, 
                             'accuracy': accuracy_list,
                             'method':method_list,
                            'tol_rate': str_tol})
df_admm.head()

,time,accuracy,method,tol_rate
0,0.009075,0.947838,admm,0.0001
1,0.010913,0.954838,admm,1e-05
2,0.012181,0.955482,admm,1e-06
3,0.012739,0.955560,admm,1e-07


# Visualization

In [45]:
frames = [df_sk, df_rg, df_admm]

df = pd.concat(frames)
df

,time,accuracy,method,tol_rate
0,0.002336,0.999993,sklearn,0.0001
1,0.001507,0.999993,sklearn,1e-05
2,0.001051,0.999993,sklearn,1e-06
3,0.001518,1.000000,sklearn,1e-07
0,0.002289,0.997916,regain,0.0001
1,0.001779,0.999697,regain,1e-05
2,0.002567,0.999976,regain,1e-06
3,0.002667,0.999999,regain,1e-07
0,0.009075,0.947838,admm,0.0001
1,0.010913,0.954838,admm,1e-05


In [46]:
#normalised accuracy [0,1]
norm_ac = np.array(df['accuracy'])
df['accuracy'] = (norm_ac - np.min(norm_ac))/np.ptp(norm_ac)
df

,time,accuracy,method,tol_rate
0,0.002336,0.999865,sklearn,0.0001
1,0.001507,0.999865,sklearn,1e-05
2,0.001051,0.999865,sklearn,1e-06
3,0.001518,1.000000,sklearn,1e-07
0,0.002289,0.960045,regain,0.0001
1,0.001779,0.994192,regain,1e-05
2,0.002567,0.999543,regain,1e-06
3,0.002667,0.999983,regain,1e-07
0,0.009075,0.000000,admm,0.0001
1,0.010913,0.134210,admm,1e-05


In [47]:
# fig = px.line(df, x="time", y="accuracy", color= "method",text="tol_rate", title='Perfomance comparison')
fig = px.line(df, x="time", y="accuracy", color= "method", title='Perfomance comparison')
fig.show()

# Import LatentTimeGraphicalLasso

In [48]:
# np.random.seed(243)

# tau = 0.1 #mu1 in ADMM_SGL
# alpha = 0.1 # lambda in ADMM_SGL

# sol = latent_graphical_lasso_.latent_graphical_lasso(
#     true_cov, alpha=alpha, tau=tau, tol=1e-5, rtol=1e-5,
#     rho=1. / true_cov.shape[0], verbose=0, max_iter=500)

# prec, cov, n_iter = sol[0], sol[1], sol[2]

# Power networks

In [9]:
p = 100
N = 200
Sigma, Theta = group_power_network(p, K=5, M=2) #Theta is true precision matrix

S, samples = sample_covariance_matrix(Sigma, N)

In [10]:
S = S[0,:,:]
Theta = Theta[0,:,:]

In [11]:
samples.shape

(5, 100, 200)

## Model Solution Z_n

In [12]:
Z = Theta
Z.shape

(100, 100)

In [47]:
start = time.time()
Z_n = sk_GL(alpha=0.01, max_iter=50000, tol = 1e-6).fit(samples[0,:,:].T)
end = time.time()

hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

00:00:00.97


In [20]:
lambda1 = 0.01
Omega_0 = np.eye(p)

start = time.time()
sol, info = ADMM_SGL(S, lambda1, Omega_0, max_iter=50000, tol = 1e-6, verbose = False, latent = False)
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

ADMM terminated after 57 iterations with accuracy 0.005571403508857344
ADMM status: {'dual optimal', 'primal optimal'}
00:00:00.22


In [26]:
start = time.time()
sol, info = kkt_ADMM_SGL(S, lambda1, Omega_0, max_iter=50000, eps_admm=1e-6, verbose = False, latent = False)
end = time.time()
hours, rem = divmod(end-start, 3600)
minutes, seconds = divmod(rem, 60)
print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

ADMM terminated after 135 iterations with accuracy 9.712746514452672e-07
ADMM status: optimal
00:00:00.88


## Sklearn

In [35]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = np.linspace(1e-10, 1e-4, num=100)
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    Z_n = sk_GL(alpha=0.01, max_iter=50000, tol = tol).fit(samples[0,:,:].T)
    end = time.time()
    hours, rem = divmod(end-start, 3600)
    minutes, seconds = divmod(rem, 60)
    time_list.append("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))
    
    cov_list.append(Z_n.covariance_)
    precision_list.append(Z_n.precision_)

In [38]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = np.linalg.norm(Z - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [39]:
method_list = ["sklearn"]*len(accuracy_list)

In [42]:
str_tol = [str(x) for x in tol_list]

In [43]:
df_sk = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method': method_list,
                            'tol_rate': str_tol})
df_sk.head()

,time,accuracy,method,tol_rate
0,00:00:00.11,11.770039,sklearn,0.0001
1,00:00:00.70,11.770009,sklearn,1e-05
2,00:00:01.43,11.770003,sklearn,1e-06
3,00:00:01.79,11.770002,sklearn,1e-07


## Regain

In [68]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = np.linspace(1e-10, 1e-4, num=100)
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = rg_GL(alpha=0.01, rho=1.,tol=tol, rtol=1e-7, max_iter=1000, verbose=False).fit(samples[0,:,:].T)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

In [69]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z_n.precision_ - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [70]:
method_list = ["regain"]*len(accuracy_list)

In [71]:
df_rg = pd.DataFrame(data={'time': time_list, 
                           'accuracy': accuracy_list,
                           'method': method_list,
                            'tol_rate': str_tol})
df_rg.head()

,time,accuracy,method,tol_rate
0,0.069482,0.991595,regain,0.0001
1,0.043811,0.999290,regain,1e-05
2,0.049675,0.999930,regain,1e-06
3,0.064751,0.999992,regain,1e-07


## ADMM

In [63]:
lambda1 = 0.01
Omega_0 = np.eye(p)

In [64]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
#     sol, info = ADMM_SGL(X_cov, lambda1, Omega_0, max_iter=1000, tol = tol, verbose = False, latent = False)
    sol, info = ADMM_SGL(S, lambda1, Omega_0, max_iter=1000, tol = tol, verbose = False, latent = False)
    end = time.time()
    cov_list.append(sol['Omega'])
    precision_list.append(sol['Theta'])
    time_list.append(end-start)

ADMM terminated after 4 iterations with accuracy 0.3214629072359936
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 10 iterations with accuracy 0.04154100149565893
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 18 iterations with accuracy 0.005343623622335427
ADMM status: {'dual optimal', 'primal optimal'}
ADMM terminated after 25 iterations with accuracy 0.001175897843920136
ADMM status: {'dual optimal', 'primal optimal'}


In [65]:
accuracy_list = []
for i in range(0, len(precision_list)):
    accuracy = 1 - np.linalg.norm(Z_n.precision_ - np.array(precision_list[i]))
    accuracy_list.append(accuracy)

In [66]:
method_list = ["admm"]*len(accuracy_list)

In [67]:
df_admm = pd.DataFrame(data={'time': time_list, 
                             'accuracy': accuracy_list,
                             'method':method_list,
                            'tol_rate': str_tol})
df_admm.head()

,time,accuracy,method,tol_rate
0,0.032021,0.233883,admm,0.0001
1,0.052513,0.862386,admm,1e-05
2,0.077470,0.977905,admm,1e-06
3,0.046352,0.994846,admm,1e-07


## Visualization

In [72]:
frames = [df_sk, df_rg, df_admm]

df = pd.concat(frames)
df

,time,accuracy,method,tol_rate
0,0.002336,0.999993,sklearn,0.0001
1,0.001507,0.999993,sklearn,1e-05
2,0.001051,0.999993,sklearn,1e-06
3,0.001518,1.000000,sklearn,1e-07
0,0.069482,0.991595,regain,0.0001
1,0.043811,0.999290,regain,1e-05
2,0.049675,0.999930,regain,1e-06
3,0.064751,0.999992,regain,1e-07
0,0.032021,0.233883,admm,0.0001
1,0.052513,0.862386,admm,1e-05


In [73]:
# fig = px.line(df, x="time", y="accuracy", color= "method",text="tol_rate", title='Perfomance comparison')
fig = px.line(df, x="time", y="accuracy", color= "method", title='Perfomance comparison')
fig.show()

## AGP data

In [74]:
import pyreadr
import pandas as pd
import numpy as np

result = pyreadr.read_r('/Users/oleg.vlasovetc/Desktop/amgut1.filt.rda')

In [80]:
data = result.values()
data = np.array(list(data))
data = pd.DataFrame(data[0, :, :])
data.shape

(289, 127)

In [79]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,117,118,119,120,121,122,123,124,125,126
0,0.0,7.0,0.0,4.0,2.0,858.0,0.0,0.0,179.0,2.0,...,22.0,0.0,1.0,0.0,0.0,2452.0,0.0,0.0,98.0,0.0
1,0.0,43.0,0.0,1.0,129.0,0.0,0.0,0.0,1963.0,0.0,...,1256.0,1.0,12.0,0.0,0.0,37.0,0.0,1.0,4.0,0.0
2,8.0,0.0,1.0,6.0,191.0,948.0,4.0,0.0,9.0,0.0,...,16.0,0.0,0.0,4.0,1.0,253.0,0.0,16.0,9.0,0.0
3,0.0,0.0,0.0,0.0,3.0,152.0,0.0,4.0,0.0,0.0,...,4.0,0.0,0.0,0.0,3.0,2129.0,0.0,2.0,111.0,0.0
4,0.0,28.0,1.0,0.0,286.0,1.0,0.0,0.0,0.0,0.0,...,2.0,9.0,0.0,0.0,0.0,330.0,4.0,0.0,24.0,5.0


In [86]:
X = np.random.multivariate_normal(mean=np.zeros(289), cov=np.cov(data), size=100)
X.shape

(100, 289)

In [87]:
tol_list = [1e-4, 1e-5, 1e-6, 1e-7]
# tol_list = np.linspace(1e-10, 1e-4, num=100)
# tol_list = [1e-1, 1e-2]
cov_list = []
precision_list = []
time_list = []
for tol in tol_list:
    start = time.time()
    rg = sk_GL(tol=tol, max_iter = 1000).fit(X)
    end = time.time()
    cov_list.append(rg.covariance_)
    precision_list.append(rg.precision_)
    time_list.append(end-start)

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 374199795537.8548, tolerance: 74878331.50470486

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: 39612000292851.42, tolerance: 7922409579.533592

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 13908230023.153254

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You 

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 31933890.51298965

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 190678822.8408454

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 618447361.9974291

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the nu

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 25566406906.572796

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 1471917169.5123873

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 3547378928.809455

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the 

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 240469955.4805804

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 29700832436.087044

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 445811454.40413314

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the 

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 70111154.8963698

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 13915615180.088654

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 54685714.38481976

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the nu

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 31378764.46369518

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 18968575548.83934

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 114230368.38614893

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the n

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 15286706300.33952

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 25830917898.156803

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 20374471406.497684

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the 

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 8764588786.623425

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 44815387642.173996

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Duality gap: nan, tolerance: 61186424815.47965

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:229: ConvergenceWarning:

Objective did not converge. You might want to increase the n

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:243: RuntimeWarning:

invalid value encountered in multiply

/Users/oleg.vlasovetc/opt/anaconda3/envs/gglasso/lib/python3.8/site-packages/sklearn/covariance/_graph_lasso.py:245: RuntimeWarning:

invalid value encountered in multiply



FloatingPointError: The system is too ill-conditioned for this solver. The system is too ill-conditioned for this solver